In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

Create model in class version

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() #RandomNormal에서 glorot_uniform으로 바꿔준다

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))   #layer
            self.model.add(batch_norm())              #norm
            self.model.add(relu())                    #activation 순서로 많이 한다

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

Create model in function version

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

Define data & Hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

Restore checkpoint & start train or test phrase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer("C:/Users/johnk/logs/")
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        #checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\johnk\anaconda3\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.7428, train_loss: 0.28225720, train_accuracy: 0.4453, test_Accuracy: 0.2919
Epoch: [ 0] [    1/  468] time: 2.3283, train_loss: 0.25231355, train_accuracy: 0.4844, test_Accuracy: 0.3693
Epoch: [ 0] [    2/  468] time: 2.9706, train_loss: 0.25954184, train_accuracy: 0.6016, test_Accuracy: 0.4940
Epoch: [ 0] [    3/  468] time: 3.6936, train_loss: 0.30196589, train_accuracy: 0.5078, test_Accuracy: 0.5703
Epoch: [ 0] [    4/  468] time: 4.4137, train_loss: 0.41343993, train_accuracy: 0.5938, test_Accuracy: 0.6177
Epoch: [ 0] [    5/  468] time: 5.1737, train_loss: 0.29752558, train_accuracy: 0.7188, test_Accuracy: 0.6579
Epoch: [ 0] [    6/  468] time: 5.8728, train_loss: 0.30029380, train_accuracy: 0.7188, test_Accuracy: 0.6904


Epoch: [ 0] [    7/  468] time: 6.5201, train_loss: 0.26532525, train_accuracy: 0.7734, test_Accuracy: 0.7231
Epoch: [ 0] [    8/  468] time: 7.0855, train_loss: 0.20599574, train_accuracy: 0.7734, test_Accuracy: 0.7439
Epoch: [ 0] [    9/  468] time: 7.7288, train_loss: 0.37121022, train_accuracy: 0.7812, test_Accuracy: 0.7468
Epoch: [ 0] [   10/  468] time: 8.5407, train_loss: 0.34790373, train_accuracy: 0.7109, test_Accuracy: 0.7571
Epoch: [ 0] [   11/  468] time: 9.4662, train_loss: 0.25151950, train_accuracy: 0.7891, test_Accuracy: 0.7676
Epoch: [ 0] [   12/  468] time: 10.1045, train_loss: 0.21456575, train_accuracy: 0.7500, test_Accuracy: 0.7743
Epoch: [ 0] [   13/  468] time: 10.6729, train_loss: 0.25255936, train_accuracy: 0.7656, test_Accuracy: 0.7788
Epoch: [ 0] [   14/  468] time: 11.2664, train_loss: 0.31861210, train_accuracy: 0.7344, test_Accuracy: 0.7933


Epoch: [ 0] [   15/  468] time: 11.8628, train_loss: 0.21317688, train_accuracy: 0.7969, test_Accuracy: 0.8142
Epoch: [ 0] [   16/  468] time: 12.4472, train_loss: 0.16168857, train_accuracy: 0.8281, test_Accuracy: 0.8250
Epoch: [ 0] [   17/  468] time: 13.0266, train_loss: 0.21051350, train_accuracy: 0.8281, test_Accuracy: 0.8314
Epoch: [ 0] [   18/  468] time: 13.6171, train_loss: 0.20278579, train_accuracy: 0.8438, test_Accuracy: 0.8337
Epoch: [ 0] [   19/  468] time: 14.1846, train_loss: 0.18353686, train_accuracy: 0.8672, test_Accuracy: 0.8352
Epoch: [ 0] [   20/  468] time: 14.8099, train_loss: 0.20651019, train_accuracy: 0.7734, test_Accuracy: 0.8398
Epoch: [ 0] [   21/  468] time: 15.4272, train_loss: 0.21536678, train_accuracy: 0.8750, test_Accuracy: 0.8432
Epoch: [ 0] [   22/  468] time: 16.0506, train_loss: 0.18884592, train_accuracy: 0.8203, test_Accuracy: 0.8464


...

Epoch: [ 0] [  431/  468] time: 287.7252, train_loss: 0.05778784, train_accuracy: 0.9688, test_Accuracy: 0.9670
Epoch: [ 0] [  432/  468] time: 288.3066, train_loss: 0.06267355, train_accuracy: 1.0000, test_Accuracy: 0.9674
Epoch: [ 0] [  433/  468] time: 288.9299, train_loss: 0.03636618, train_accuracy: 0.9844, test_Accuracy: 0.9673
Epoch: [ 0] [  434/  468] time: 289.8126, train_loss: 0.09098773, train_accuracy: 0.9844, test_Accuracy: 0.9679
Epoch: [ 0] [  435/  468] time: 290.4938, train_loss: 0.12976857, train_accuracy: 0.9609, test_Accuracy: 0.9689
Epoch: [ 0] [  436/  468] time: 291.1101, train_loss: 0.10462765, train_accuracy: 0.9609, test_Accuracy: 0.9695
Epoch: [ 0] [  437/  468] time: 291.7534, train_loss: 0.07194401, train_accuracy: 0.9766, test_Accuracy: 0.9694
Epoch: [ 0] [  438/  468] time: 292.8844, train_loss: 0.02134144, train_accuracy: 1.0000, test_Accuracy: 0.9695


Epoch: [ 0] [  439/  468] time: 293.9286, train_loss: 0.11046757, train_accuracy: 0.9844, test_Accuracy: 0.9706
Epoch: [ 0] [  440/  468] time: 294.8641, train_loss: 0.05980483, train_accuracy: 0.9766, test_Accuracy: 0.9710
Epoch: [ 0] [  441/  468] time: 295.6500, train_loss: 0.10751246, train_accuracy: 0.9609, test_Accuracy: 0.9711
Epoch: [ 0] [  442/  468] time: 296.3162, train_loss: 0.08836444, train_accuracy: 0.9844, test_Accuracy: 0.9703
Epoch: [ 0] [  443/  468] time: 297.0502, train_loss: 0.08441643, train_accuracy: 0.9766, test_Accuracy: 0.9705
Epoch: [ 0] [  444/  468] time: 297.7483, train_loss: 0.07882835, train_accuracy: 0.9688, test_Accuracy: 0.9702
Epoch: [ 0] [  445/  468] time: 298.5193, train_loss: 0.06235561, train_accuracy: 0.9844, test_Accuracy: 0.9696
Epoch: [ 0] [  446/  468] time: 299.2154, train_loss: 0.09159292, train_accuracy: 0.9766, test_Accuracy: 0.9710


Epoch: [ 0] [  447/  468] time: 299.8737, train_loss: 0.12397486, train_accuracy: 0.9688, test_Accuracy: 0.9710
Epoch: [ 0] [  448/  468] time: 300.6187, train_loss: 0.03348276, train_accuracy: 1.0000, test_Accuracy: 0.9715
Epoch: [ 0] [  449/  468] time: 301.2769, train_loss: 0.05287047, train_accuracy: 0.9688, test_Accuracy: 0.9716
Epoch: [ 0] [  450/  468] time: 301.9790, train_loss: 0.02697268, train_accuracy: 0.9922, test_Accuracy: 0.9710
Epoch: [ 0] [  451/  468] time: 302.6931, train_loss: 0.10410638, train_accuracy: 0.9766, test_Accuracy: 0.9699
Epoch: [ 0] [  452/  468] time: 303.5359, train_loss: 0.05803633, train_accuracy: 0.9609, test_Accuracy: 0.9694
Epoch: [ 0] [  453/  468] time: 304.4414, train_loss: 0.03604981, train_accuracy: 1.0000, test_Accuracy: 0.9673
Epoch: [ 0] [  454/  468] time: 305.4068, train_loss: 0.08424609, train_accuracy: 0.9766, test_Accuracy: 0.9663


Epoch: [ 0] [  455/  468] time: 306.1618, train_loss: 0.16969512, train_accuracy: 0.9531, test_Accuracy: 0.9667
Epoch: [ 0] [  456/  468] time: 306.9756, train_loss: 0.05484148, train_accuracy: 0.9922, test_Accuracy: 0.9669
Epoch: [ 0] [  457/  468] time: 307.7386, train_loss: 0.07775727, train_accuracy: 0.9844, test_Accuracy: 0.9668
Epoch: [ 0] [  458/  468] time: 308.4198, train_loss: 0.07869919, train_accuracy: 0.9766, test_Accuracy: 0.9658
Epoch: [ 0] [  459/  468] time: 309.1768, train_loss: 0.12756209, train_accuracy: 0.9766, test_Accuracy: 0.9659
Epoch: [ 0] [  460/  468] time: 310.1252, train_loss: 0.04210255, train_accuracy: 0.9844, test_Accuracy: 0.9659
Epoch: [ 0] [  461/  468] time: 311.0128, train_loss: 0.08709763, train_accuracy: 0.9688, test_Accuracy: 0.9663
Epoch: [ 0] [  462/  468] time: 312.0421, train_loss: 0.03738388, train_accuracy: 0.9766, test_Accuracy: 0.9662


Epoch: [ 0] [  463/  468] time: 312.9257, train_loss: 0.05198143, train_accuracy: 0.9844, test_Accuracy: 0.9662
Epoch: [ 0] [  464/  468] time: 313.8094, train_loss: 0.06322785, train_accuracy: 0.9688, test_Accuracy: 0.9660
Epoch: [ 0] [  465/  468] time: 314.7259, train_loss: 0.04344005, train_accuracy: 0.9844, test_Accuracy: 0.9662
Epoch: [ 0] [  466/  468] time: 315.6295, train_loss: 0.02599688, train_accuracy: 0.9922, test_Accuracy: 0.9657
Epoch: [ 0] [  467/  468] time: 316.4214, train_loss: 0.07680932, train_accuracy: 0.9844, test_Accuracy: 0.9656


In [ ]:
 Test accuracy = 96.5% 